<table align="left"><td><a target="_blank" href="https://colab.research.google.com/github/superannotateai/model-deployment-tutorials/blob/main/JETSON/SuperAnnotate_JETSON_Yolact_edge_Deployment.ipynb"><img src="https://user-images.githubusercontent.com/25985824/104791629-6e618700-5769-11eb-857f-d176b37d2496.png" height="32" width="32"> Try in Google Colab</a></td></table>

# 0. Preparation Steps
Before starting the pipeline you need to download tgz archives of TensorRT and CUDNN and place them in your google drive. (This notebook is tested with TensorRT 7.2.3.4 and CUDNN-11.2 versions)

# 1. Install Prerequisites 
(Please click on **RESTART RUNTIME** button when it appears in the output of this code block)



In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')


In [ ]:
!cp /content/drive/MyDrive/TensorRT-7.2.3.4.Ubuntu-18.04.x86_64-gnu.cuda-11.0.cudnn8.1.tar.gz /content/
!cp /content/drive/MyDrive/cudnn-11.2-linux-x64-v8.1.1.33.tgz /content/

!tar -xzvf cudnn-11.2-linux-x64-v8.1.1.33.tgz

!sudo cp cuda/include/cudnn*.h /usr/local/cuda/include 
!sudo cp -P cuda/lib64/libcudnn* /usr/local/cuda/lib64 
!sudo chmod a+r /usr/local/cuda/include/cudnn*.h /usr/local/cuda/lib64/libcudnn*

!tar xzvf TensorRT-7.2.3.4.Ubuntu-18.04.x86_64-gnu.cuda-11.0.cudnn8.1.tar.gz
os.environ['LD_LIBRARY_PATH'] += ':/content/TensorRT-7.2.3.4/lib/'

%cd /content/TensorRT-7.2.3.4/python/
!sudo pip3 install tensorrt-7.2.3.4-cp37-none-linux_x86_64.whl

%cd /content/TensorRT-7.2.3.4/uff/
!sudo pip3 install uff-0.6.9-py2.py3-none-any.whl

%cd /content/TensorRT-7.2.3.4/graphsurgeon/
!sudo pip3 install graphsurgeon-0.4.5-py2.py3-none-any.whl
%cd /content/TensorRT-7.2.3.4/onnx_graphsurgeon
!sudo pip3 install onnx_graphsurgeon-0.2.6-py2.py3-none-any.whl

%cd /content/
!git clone https://github.com/haotian-liu/yolact_edge.git
!git clone https://github.com/NVIDIA-AI-IOT/torch2trt

!pip install gitpython
!pip install superannotate
!pip install google-resumable-media==0.5.0
!pip install onnx-simplifier

!sudo apt-get update
!sudo apt-get install libhdf5-serial-dev hdf5-tools libhdf5-dev zlib1g-dev zip libjpeg8-dev liblapack-dev libblas-dev gfortran
!sudo apt-get install python3-pip
!pip3 install -U pip testresources setuptools==49.6.0
!pip3 install future==0.18.2 mock==3.0.5 h5py==2.10.0 keras_preprocessing==1.1.1 keras_applications==1.0.8 gast==0.2.2 futures protobuf pybind11
!pip3 install --pre --extra-index-url https://developer.download.nvidia.com/compute/redist/jp/v43 'tensorflow<2'
!pip3 install pycuda

# 2. Setup your SuperAnnotate token, project names of trianing images and desired output model name

In [ ]:
TOKEN = "For Pro Users, the Token-ID can be generated in the teams section"
PROJECT_NAME = "City"
OUTPUT_MODEL = "sa-tutorial"

# 3. Download and preprocess the data for training

In [ ]:
import superannotate as sa
import json
import os 
from shutil import copy, make_archive
import re

os.environ['CUDA_INSTALL_DIR']= "/usr/local/cuda/"
os.environ['CUDNN_INSTALL_DIR']= "/usr/local/cuda/"
os.environ['LD_LIBRARY_PATH']+= ":/usr/lib64-nvidia/:/content/TensorRT-7.2.3.4/include/:/content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/:/content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/bin/"
os.environ['PATH']+= ":/content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib:/content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/bin/"
!sudo rm /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/libnvinfer.so.7
!sudo ln -s /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/libnvinfer.so.7.2.3 /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/libnvinfer.so.7
!sudo cp -r /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/*.* /usr/lib/ 
!sudo cp -r /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/bin/*.* /usr/bin/ 
!sudo cp -r /content/TensorRT-7.2.3.4/include/* /usr/local/cuda/targets/x86_64-linux/include/
!sudo cp -r /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/* /usr/local/cuda/targets/x86_64-linux/lib/

proj_folder = dict(zip(PROJECT_NAME, ["proj_" + str(i) for i in range(len(PROJECT_NAME))]))
token_json = {"token": TOKEN}
with open('sa_config.json', 'w') as f:
  json.dump(token_json, f)

sa.init('sa_config.json')

dataset_base_dir = '/content/data'
if not os.path.exists(dataset_base_dir):
  os.mkdir(dataset_base_dir)

export = sa.prepare_export(PROJECT_NAME, annotation_statuses=["Completed"], include_fuse=True)
sa.download_export(PROJECT_NAME, export, dataset_base_dir)
sa.export_annotation("/content/data", "/content/data_coco", "COCO", PROJECT_NAME, project_type="Pixel", task="instance_segmentation")

with open('/content/data/classes/classes.json', 'r') as f:
  classes_data = json.load(f)

class_names = []
label_map = {}
for i, class_data in enumerate(classes_data):
  class_names.append(class_data["name"])
  label_map[class_data["id"]] = i + 1

dataset_name = "Custom train"
img_paths = '/content/data_coco/image_set/'
annot_path = '/content/data_coco/' + PROJECT_NAME + '.json'
dataset_dict = {
  'name': dataset_name,
  'train_images': img_paths,
  'train_info':   annot_path,
  'valid_images': img_paths,
  'valid_info':   annot_path,
  'has_gt': True,
  'class_names': tuple(class_names),
  'label_map': label_map,
}

yolact_conf_dict = {
  'dataset': 'my_custom_dataset',
  'num_classes': len(class_names) + 1,
  'name': 'yolact_edge_mobilenetv2_custom',
  'backbone': 'mobilenetv2_backbone'
}
dataset_line = "my_custom_dataset = dataset_base.copy(" + str(dataset_dict) + ")"

yolact_conf_line = "yolact_edge_config_custom = yolact_edge_config.copy(" + str(yolact_conf_dict) + ")"

with open('/content/yolact_edge/data/config.py', 'a') as f:
  f.write('\n')
  f.write(dataset_line)
  f.write('\n')
  f.write(yolact_conf_line)

label_map = "'label_map': " + str(label_map)
with open('/content/yolact_edge/data/config.py') as f:
    s = f.read()

s = re.sub("'label_map': None", label_map, s)
s = re.sub("'label_map': COCO_LABEL_MAP", label_map, s)
s = re.sub("'dataset': 'my_custom_dataset'","'dataset': my_custom_dataset", s)
s = re.sub("'backbone': 'mobilenetv2_backbone'","'backbone': mobilenetv2_backbone", s)

with open('/content/yolact_edge/data/config.py', 'w') as f:
  f.write(s)

# 4. Train the model

In [ ]:
!mkdir /content/yolact_edge/weights
%cd /content/yolact_edge/weights
!wget 'https://download.pytorch.org/models/mobilenet_v2-b0353104.pth'

%cd /content/yolact_edge/

!python train.py --config=yolact_edge_config_custom --dataset=my_custom_dataset --num_gpus 1

# 5. Generate and download the files needed for the Jetson device

In [ ]:
from google.colab import files
%cd /content/torch2trt
!sudo python setup.py install --plugins

%cd /content/yolact_edge/
!wget https://c402277.ssl.cf1.rackcdn.com/photos/18315/images/hero_small/Medium_WW230176.jpg
!python eval.py --config=yolact_edge_config_custom\
 --trained_model="latest"\
 --image="Medium_WW230176.jpg"\
 --use_fp16_tensorrt\
 --use_tensorrt_safe_mode
 
!zip -r "{OUTPUT_MODEL}.zip" /content/yolact_edge/weights/*.trt
files.download(OUTPUT_MODEL + '.zip')

# 6. Test the custom model

Afterwards you can load the files in the archive with tensorrt on jetson devices and integrate it to your pipelines.
